In [16]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms 
import torchvision.models as models
from torchvision.utils import save_image

In [17]:
  model=models.vgg19(pretrained=True)

In [18]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [19]:
class VGG(nn.Module):
    def __init__(self):
      super(VGG,self).__init__()
      self.choosen_features =['0','5','10','19','28']
      self.model =models.vgg19(pretrained=True).features[:29]
    
    def forward(self,x):
      features=[]
      for layer_num,layer in enumerate(self.model):
          x=layer(x)

          if str(layer_num) in self.choosen_features:
            features.append(x)
      return features

In [20]:
def load_image (image_name):
  image=Image.open(image_name)
  image=loader(image).unsqueeze(0)

  return image.to(device)

In [21]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size=356
loader= transforms.Compose ([
                    transforms.Resize((image_size,image_size)),
                    transforms.ToTensor()
                          ])


In [22]:
original_img=load_image(r"/content/pp.jpeg")
style_img=load_image(r"/content/style_4.jpg")
model= VGG().to(device)

In [23]:
#generated =torch.randn(original_img.shape ,device=device,requires_grad=True)
generated =original_img.clone().requires_grad_(True)

In [24]:
%%time
total_steps=6000
learning_rate =0.001
alpha =1
beta=0.01

optimizer=optim.Adam([generated], lr=learning_rate)

for step in range(total_steps):
    generated_features= model(generated)
    original_img_features = model(original_img)
    style_features =model(style_img)

    style_loss=original_loss=0

    for gen_feature ,orig_feature,style_feature in zip(
        generated_features, original_img_features, style_features ):
      
              batch_size,channel,height,width=gen_feature.shape

              original_loss +=torch.mean((gen_feature - orig_feature)**2)

              G= gen_feature.view(channel,height*width).mm(
                  gen_feature.view(channel,height*width).t()
              )

              A=style_feature.view(channel,height*width).mm(
                  style_feature.view(channel,height*width).t()
              )
              style_loss +=torch.mean((G-A)**2)

    total_loss= alpha* original_loss + beta *style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step% 200 ==0:
        print(total_loss)
        save_image(generated,"generated.png")


tensor(164396.0938, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8771.5244, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3776.9280, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2362.3718, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1844.7393, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1557.1384, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1356.5251, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1202.5973, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1076.7419, device='cuda:0', grad_fn=<AddBackward0>)
tensor(971.4896, device='cuda:0', grad_fn=<AddBackward0>)
tensor(882.4050, device='cuda:0', grad_fn=<AddBackward0>)
tensor(805.0089, device='cuda:0', grad_fn=<AddBackward0>)
tensor(737.2241, device='cuda:0', grad_fn=<AddBackward0>)
tensor(677.3459, device='cuda:0', grad_fn=<AddBackward0>)
tensor(624.3973, device='cuda:0', grad_fn=<AddBackward0>)
tensor(577.0915, device='cuda:0', grad_fn=<AddBackward0>)
tensor(534.7161, device='cuda:0', grad_fn=<AddBackward0>)
ten

In [25]:
# for step in range(total_steps):
#     # Obtain the convolution features in specifically chosen layers
#     generated_features = model(generated)
#     original_img_features = model(original_img)
#     style_features = model(style_img)

#     # Loss is 0 initially
#     style_loss = original_loss = 0

#     # iterate through all the features for the chosen layers
#     for gen_feature, orig_feature, style_feature in zip(
#         generated_features, original_img_features, style_features
#     ):

#         # batch_size will just be 1
#         batch_size, channel, height, width = gen_feature.shape
#         original_loss += torch.mean((gen_feature - orig_feature) ** 2)
#         # Compute Gram Matrix of generated
#         G = gen_feature.view(channel, height * width).mm(
#             gen_feature.view(channel, height * width).t()
#         )
#         # Compute Gram Matrix of Style
#         A = style_feature.view(channel, height * width).mm(
#             style_feature.view(channel, height * width).t()
#         )
#         style_loss += torch.mean((G - A) ** 2)

#     total_loss = alpha * original_loss + beta * style_loss
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()

#     if step % 200 == 0:
#         print(total_loss)
#         save_image(generated, "generated.png")